# 基于 Chatterbot 制作中文聊天机器人

## 手动设置一点语料

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

Chinese_bot = ChatBot("Training demo") #创建一个新的实例
Chinese_bot.set_trainer(ListTrainer)
Chinese_bot.train([
    '亲，在吗？',
    '亲，在呢',
    '这件衣服的号码大小标准吗？',
    '亲，标准呢，请放心下单吧。',
    '有红色的吗？',
    '有呢，目前有白红蓝3种色调。',
])

List Trainer: [####################] 100%


测试一下效果

In [2]:
question = '亲，在吗'
print(question)
Chinese_bot.get_response(question)

亲，在吗


<Statement text:亲，在呢>

In [3]:
question = '有红色的吗？'
print(question)
Chinese_bot.get_response(question)

有红色的吗？


<Statement text:有呢，目前有白红蓝3种色调。>

## 训练自己的语料

### 载入语料

In [5]:
lines = open("data/QQ.txt", "r", encoding="GBK").readlines()
sec = [line.strip() for line in lines]

## 训练模型

In [6]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

In [11]:
Chinese_bot = ChatBot("Training")
Chinese_bot.set_trainer(ListTrainer)
Chinese_bot.train(sec[:5000])

List Trainer: [####################] 100%


In [12]:
Chinese_bot.get_response("努力学习")

<Statement text:062015:爬虫>

In [13]:
Chinese_bot.get_response("做开发啊")

<Statement text:我当时就是清华实验室做事啊>

# 基于 Seq2Seq 制作中文聊天机器人

## 语料准备

用Tab键\t把问题和答案区分

## 模型构建和训练

In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

Using TensorFlow backend.


### 定义模型超参数、迭代次数、语料路径

In [19]:
#Batch size 的大小
batch_size = 32  
# 迭代次数epochs
epochs = 10
# 编码空间的维度Latent dimensionality 
latent_dim = 256  
# 要训练的样本数
num_samples = 5000 
#设置语料的路径
data_path = "data/files.txt"

### 语料向量化

In [20]:
# 数据向量化
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    #print(line)
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = target_text[0: 100]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Number of samples: 4999
Number of unique input tokens: 1428
Number of unique output tokens: 1970
Max sequence length for inputs: 41
Max sequence length for outputs: 61


### LSTM_Seq2Seq 模型定义、训练和保存

In [21]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 输出 `encoder_outputs` 
encoder_states = [state_h, state_c]

# 状态 `encoder_states` 
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                       initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定义模型
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 训练
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# 保存模型
model.save('data/s2s.h5')

Train on 3999 samples, validate on 1000 samples
Epoch 1/10
3999/3999 [==============================] - 151s 38ms/step - loss: 1.6785 - val_loss: 1.7395
Epoch 2/10
3999/3999 [==============================] - 149s 37ms/step - loss: 1.6297 - val_loss: 1.7084
Epoch 3/10
3999/3999 [==============================] - 156s 39ms/step - loss: 1.5762 - val_loss: 1.6429
Epoch 4/10
3999/3999 [==============================] - 162s 41ms/step - loss: 1.4926 - val_loss: 1.5595
Epoch 5/10
3999/3999 [==============================] - 158s 39ms/step - loss: 1.4092 - val_loss: 1.4919
Epoch 6/10
3999/3999 [==============================] - 162s 41ms/step - loss: 1.3459 - val_loss: 1.4539
Epoch 7/10
3999/3999 [==============================] - 153s 38ms/step - loss: 1.2977 - val_loss: 1.4188
Epoch 8/10
3999/3999 [==============================] - 148s 37ms/step - loss: 1.2642 - val_loss: 1.3890
Epoch 9/10
3999/3999 [==============================] - 149s 37ms/step - loss: 1.2254 - val_loss: 1.3648
Epoch 1

C:\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Seq2Seq 的 Encoder 操作

In [22]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

### 把索引和分词转成序列

In [23]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### 定义预测函数，先使用预模型预测，然后编码成汉字结果

In [24]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    #print(states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        # 更新状态
        states_value = [h, c]
    return decoded_sentence

## 模型预测

In [25]:
def predict_ans(question):
    inseq = np.zeros((len(question), max_encoder_seq_length, num_encoder_tokens),dtype='float16')
    decoded_sentence = decode_sequence(inseq)
    return decoded_sentence

In [26]:
print('Decoded sentence:', predict_ans("挖机履带掉了怎么装上去"))

Decoded sentence: 油封油封油封油封油封油封油封漏油管子



训练时间较短，结果很烂，so，有机会做100轮训练试试